In [1]:
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from dotenv import load_dotenv
import os
load_dotenv()
groq_key = os.getenv("GROQ")
groq_key


'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'

In [2]:
groq_model = ChatGroq(
    model="llama3-70b-8192",
    groq_api_key=groq_key)

In [9]:
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(
    "Tell me a curious fact about {something}.",
)
traditional_chain = LLMChain(
    llm=groq_model,
    prompt=prompt
    )
## Old method
traditional_chain.invoke("India")

{'something': 'India',
 'text': "Here's one:\n\nIndia has a village called Shani Shingnapur, where houses do not have doors or locks! Yes, you read that right! This village in Maharashtra, India, has a population of around 3,000 people, and none of the houses have doors or locks. The villagers believe that the village is protected by the blessings of Shani Dev, the Hindu god of Saturn, and that no thief would dare to steal from the village. And, amazingly, there have been no reported cases of theft or burglary in the village for centuries!"}

In [11]:
## LCEL
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(
    "Tell me a curious fact about {something}.",
)

In [12]:
chain_lcel = prompt | groq_model | StrOutputParser()
chain_lcel.invoke("India")

'Here\'s one:\n\nIndia has a village called Shani Shingnapur, located in the state of Maharashtra, where there are NO DOORS OR LOCKS on any of the houses! Yes, you read that right! For centuries, the villagers have lived without doors or locks, and yet, there has never been a single reported case of theft or burglary in the village. The villagers believe that the village is protected by the blessings of Shani Dev, the Hindu god of Saturn, and that anyone who tries to steal will be cursed. This unique phenomenon has made Shani Shingnapur a popular tourist destination, with visitors flocking to see the "doorless" houses!'

## RunnablePassthrough
#### It does not do anything to the input data
#### A chain with just RunnablePassthrough() will output the original input without anu modification

In [13]:
from langchain_core.runnables import RunnablePassthrough
chain = RunnablePassthrough()

In [14]:
chain.invoke("India")

'India'

## RunnableLambda
#### To use a custom function inside a LCEL chain we need to wrap it up with RunnableLambda
#### Let's define a very simple function to create Russian Lastnames

In [16]:
def russian_lastname(name: str) -> str:
    return f"{name}ovich"

In [18]:
from langchain_core.runnables import RunnableLambda
chain = RunnablePassthrough() | RunnableLambda(russian_lastname)

In [19]:
chain.invoke("John Doe")

'John Doeovich'

## RunnableParallel

#### We use this to run parallel tasks
#### Most important and useful runnable in langchain
#### Two tasks: 
##### operation_a will use RunnablePassthrough
##### operation_b will use RunnableLambda with the russian_lastname function


In [20]:
from langchain_core.runnables import RunnableParallel
chain = RunnableParallel(
    {
        "operation_a": RunnablePassthrough(),
        "operation_b": RunnableLambda(russian_lastname)
    }
)

In [21]:
chain.invoke("John Doe")

{'operation_a': 'John Doe', 'operation_b': 'John Doeovich'}

In [22]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [23]:
prompt = ChatPromptTemplate.from_template(
    "Tell me a curious fact about {person_name}"
)
output_parser = StrOutputParser()

In [37]:
def russian_lastname_from_dictionary(person):
    return person['person_name'] + "ovich"

In [40]:
chain = RunnableParallel(
    {
        
        "operation_b": RunnablePassthrough(),
        "person_name": RunnableLambda(russian_lastname_from_dictionary),
        "operation_c": RunnablePassthrough()
    } 
) | prompt | groq_model | output_parser
print(chain.invoke({"person_name": "Micky Mouse"}))

I apologize, but there is no such person as Micky Mouseovich. Mickey Mouse is a fictional character created by Walt Disney and Ub Iwerks in 1928. He is a cartoon character and not a real person, so there is no curious fact about a person named Micky Mouseovich.

However, if you're interested, I can share a curious fact about Mickey Mouse! Did you know that Mickey Mouse was originally going to be named "Mortimer Mouse" before Walt Disney's wife, Lillian, convinced him to change the name to Mickey Mouse?


In [41]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.embeddings import HuggingFaceEmbeddings

In [43]:
vectorstore = FAISS.from_texts(
    [
        "Learning logic ficuses on providing content on Data Science, AI, ML, DL, CV, NLP, Python Programming, etc. in English."
    ],
    embedding=HuggingFaceEmbeddings(),
)

C:\Users\Bichu George\AppData\Local\Temp\ipykernel_10508\2264838036.py:5: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedding=HuggingFaceEmbeddings(),
c:\Users\Public\anaconda3\envs\langchaingroq\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Public\anaconda3\envs\langchaingroq\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Bichu George\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded ve

In [45]:
retriever = vectorstore.as_retriever()
template = """
 Answer the question based only on the following context: {context}
 Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)


In [48]:
retriver_chain = (
    RunnableParallel(
        {
            "context": retriever,
            "question": RunnablePassthrough(),
        }
    )
    | prompt
    | groq_model
    | StrOutputParser()
)

retriver_chain.invoke("What is learning logic?")

'Based on the provided context, Learning Logic is focused on providing content on various topics such as Data Science, AI, ML, DL, CV, NLP, and Python Programming, all in the English language.'